In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### Reading the data

In [2]:
data_march_1 = pd.read_csv('march_riel_1.csv')
data_march_2 = pd.read_csv('march_riel_2.csv')
data_april = pd.read_csv('april_riel.csv')
data_may = pd.read_csv('may_riel.csv')

In [3]:
data_march_2.head()

,district,wall_material,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,type_build,price
0,Личаківський,цегла,відмінний,1.0,11.0,27.0,52,1,9,11,NaN,12500
1,Личаківський,цегла,NaN,2.0,6.0,42.0,58,3,5,5,Хрущовка,8500
2,Франківський,цегла,відмінний,1.0,18.0,15.0,44,1,2,3,Польський,19300
3,Франківський,NaN,NaN,1.0,NaN,NaN,48,1,8,10,NaN,15000
4,Галицький,NaN,люкс,1.0,NaN,NaN,50,1,8,12,NaN,30800


In [4]:
data_april.head()

,district,wall_material,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,type_build,price
0,Личаківський,цегла,люкс,1.0,15.0,NaN,52,2,2,3,NaN,19300
1,Галицький,цегла,люкс,1.0,NaN,35.0,45,1,9,12,NaN,28900
2,Франківський,NaN,NaN,NaN,25.0,15.0,45,1,8,9,NaN,17300
3,Франківський,цегла,відмінний,1.0,18.0,15.0,44,1,2,3,Польський,19300
4,Франківський,NaN,відмінний,NaN,NaN,NaN,83,2,4,10,NaN,34600


In [5]:
data_may.head()

,district,wall_material,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,type_build,price
0,Галицький,цегла,люкс,1.0,7.0,30.0,46,2,3,5,NaN,16000
1,Сихівський,NaN,люкс,1.0,NaN,NaN,31,1,5,9,NaN,10000
2,Личаківський,NaN,відмінний,1.0,10.0,NaN,75,2,3,3,Австрійський,30800
3,Шевченківський,цегла,люкс,NaN,20.0,12.0,45,1,1,9,NaN,23100
4,Личаківський,цегла,люкс,1.0,13.0,NaN,52,2,2,3,NaN,15400


In [6]:
data_march_2.shape

(3927, 12)

In [7]:
data_april.shape

(3927, 12)

In [8]:
data_may.shape

(3924, 12)

In [9]:
data_march_1.shape

(4800, 10)

### Reindex each dataset for concat

In [10]:
data_april.index = np.arange(len(data_march_2)+1, len(data_april) + len(data_march_2) + 1)

In [11]:
full_data = pd.concat([data_march_2, data_april])

In [12]:
data_may.index = np.arange(len(full_data)+1, len(data_may) + len(full_data) + 1)

In [13]:
full_data = pd.concat([full_data, data_may])

### Drop duplicates

In [14]:
full_data = full_data.drop_duplicates()

### Data cleaning

In [15]:
full_data.isna().sum()

district             0
wall_material     4275
status            2060
balcony           4094
area_kitchen      4346
area_living       5088
area                 0
room                 0
floor                0
superficiality       0
type_build        5500
price                0
dtype: int64

In [16]:
full_data.shape

(6826, 12)

According to the terms of the website: 
 - if data doesn't have a value about information *number of balcony* this mean that number of balcony is 0 
 - *building type* is new building

In [17]:
full_data.balcony = full_data.balcony.fillna(0)

In [18]:
full_data.type_build.unique()

array([nan, ' Хрущовка', ' Польський', ' Чешка', ' Австрійський',
       ' Польський люкс', ' Австрійський люкс', ' Будівля старого Львова',
       ' Гуртожиток', ' Сталінка', ' Готелька', ' Брежнєвка', ' Особняк'],
      dtype=object)

In [19]:
full_data = full_data.loc[full_data.type_build != ' Особняк']

In [20]:
full_data.type_build = full_data.type_build.fillna('new')

#### Group data by categories 

In [21]:
replacements = {
                'new': 'new', 
                ' Хрущовка': 'old', 
                ' Чешка': 'old', 
                ' Гуртожиток': 'old', 
                ' Сталінка': 'old', 
                ' Готелька': 'old', 
                ' Брежнєвка': 'old'
}
full_data['age'] = full_data.type_build.map(replacements).fillna('his')

Data has more than 50 % missing values in collumn *'wall_materials'* so we need to drop these values

In [22]:
full_data = full_data.drop(columns = ['wall_material'], axis = 1)

In [23]:
full_data.isna().sum()

district             0
status            2048
balcony              0
area_kitchen      4323
area_living       5059
area                 0
room                 0
floor                0
superficiality       0
type_build           0
price                0
age                  0
dtype: int64

In [24]:
full_data.head()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,type_build,price,age
0,Личаківський,відмінний,1.0,11.0,27.0,52,1,9,11,new,12500,new
1,Личаківський,NaN,2.0,6.0,42.0,58,3,5,5,Хрущовка,8500,old
2,Франківський,відмінний,1.0,18.0,15.0,44,1,2,3,Польський,19300,his
3,Франківський,NaN,1.0,NaN,NaN,48,1,8,10,new,15000,new
4,Галицький,люкс,1.0,NaN,NaN,50,1,8,12,new,30800,new


#### Transform categorial str data to int

In [25]:
full_data.district.unique()

array(['Личаківський', 'Франківський', 'Галицький', 'Шевченківський',
       'Сихівський', 'Залізничний'], dtype=object)

In [26]:
string_to_number_district = {
                         'Личаківський': 1, 
                         'Галицький': 2,  
                         'Шевченківський': 3, 
                         'Сихівський': 4,
                         'Франківський': 5, 
                         'Залізничний': 6}

In [27]:
full_data['district'] = full_data.district.map(string_to_number_district)

In [28]:
full_data.head()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,type_build,price,age
0,1,відмінний,1.0,11.0,27.0,52,1,9,11,new,12500,new
1,1,NaN,2.0,6.0,42.0,58,3,5,5,Хрущовка,8500,old
2,5,відмінний,1.0,18.0,15.0,44,1,2,3,Польський,19300,his
3,5,NaN,1.0,NaN,NaN,48,1,8,10,new,15000,new
4,2,люкс,1.0,NaN,NaN,50,1,8,12,new,30800,new


In [71]:
string_to_number_age = {
                         'old': 1, 
                         'his': 2,  
                         'new': 3
}

In [30]:
full_data.age = full_data.age.map(string_to_number_age)

In [31]:
full_data = full_data.drop(columns = ['type_build'], axis = 1)

In [32]:
full_data.isna().sum()

district             0
status            2048
balcony              0
area_kitchen      4323
area_living       5059
area                 0
room                 0
floor                0
superficiality       0
price                0
age                  0
dtype: int64

 - *status* column is important column for *price* prediction, let's look for price range for each categories *'status'* 
 - before filling missed data *'status'*, work with data outliers

In [33]:
full_data.describe().round()

,district,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,6778.0,6778.0,2455.0,1719.0,6778.0,6778.0,6778.0,6778.0,6778.0,6778.0
mean,3.0,0.0,17.0,36.0,65.0,2.0,5.0,8.0,30073.0,3.0
std,1.0,1.0,54.0,33.0,244.0,1.0,3.0,4.0,226838.0,1.0
min,1.0,0.0,3.0,6.0,7.0,1.0,1.0,1.0,19.0,1.0
25%,2.0,0.0,9.0,20.0,44.0,1.0,2.0,5.0,13500.0,3.0
50%,3.0,0.0,13.0,32.0,52.0,2.0,4.0,9.0,17000.0,3.0
75%,5.0,1.0,17.0,42.0,70.0,2.0,7.0,10.0,21200.0,3.0
max,6.0,4.0,1628.0,654.0,15000.0,7.0,21.0,25.0,9801000.0,3.0


Rent price max is 9801000, it's a big price for rent apartment, so drop all values where price > 100000 for month

In [34]:
full_data = full_data.loc[full_data.price < 100000]

The minimum rent price is 17, it's a measly price for rent apartment.
According to the web site, we know that price rent apartments start from 7000/month, 
so values < 7000/month are outliers: this data is likely to contain data about renting a room, not an apartment

In [35]:
full_data = full_data.loc[full_data.price >= 7000]

In [36]:
full_data.status.unique()

array([' відмінний', nan, ' люкс', ' середній', ' без оздоблення',
       ' потребує ремонту'], dtype=object)

In [37]:
full_data.shape

(6548, 11)

In [38]:
full_data.loc[full_data.status == ' без оздоблення']

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
1508,3,без оздоблення,1.0,12.0,30.0,50,2,2,9,13500,3


In [39]:
full_data = full_data.loc[full_data.status != ' без оздоблення']

In [40]:
full_data.loc[full_data.status == ' потребує ремонту']

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
7846,3,потребує ремонту,0.0,NaN,NaN,64,2,2,10,22000,3
9360,2,потребує ремонту,1.0,5.0,28.0,45,2,2,2,10000,2
11501,5,потребує ремонту,2.0,NaN,NaN,75,4,7,9,17300,3


We have only one instance with *status* ' без оздоблення' and three ' потребує ремонту': remove the data for the representativeness of the sample

In [41]:
full_data = full_data.loc[full_data.status != ' потребує ремонту']

In [42]:
full_data.loc[full_data.status.isna()].describe().round()

,district,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,1965.0,1965.0,260.0,192.0,1965.0,1965.0,1965.0,1965.0,1965.0,1965.0
mean,4.0,0.0,13.0,37.0,66.0,2.0,5.0,8.0,17712.0,3.0
std,1.0,0.0,8.0,30.0,339.0,1.0,3.0,4.0,8181.0,0.0
min,1.0,0.0,4.0,11.0,9.0,1.0,1.0,1.0,7000.0,1.0
25%,2.0,0.0,8.0,25.0,43.0,1.0,2.0,5.0,12000.0,3.0
50%,3.0,0.0,12.0,33.0,50.0,2.0,4.0,9.0,16000.0,3.0
75%,5.0,0.0,15.0,41.0,65.0,2.0,6.0,10.0,21200.0,3.0
max,6.0,3.0,73.0,382.0,15000.0,7.0,21.0,25.0,76900.0,3.0


In [43]:
full_data.status.unique()

array([' відмінний', nan, ' люкс', ' середній'], dtype=object)

Transform categories data str to int

In [72]:
string_to_number_status = {
                         ' середній': 1, 
                         ' відмінний': 2,  
                         ' люкс': 3
}

In [45]:
full_data.status = full_data.status.map(string_to_number_status)

#### Overview the data with each category *status* 

In [46]:
full_data.loc[full_data.status == 1].describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,292.0,292.0,292.0,133.0,93.0,292.0,292.0,292.0,292.0,292.0,292.0
mean,4.0,1.0,1.0,11.0,34.0,52.0,2.0,4.0,6.0,11623.0,2.0
std,2.0,0.0,1.0,10.0,32.0,28.0,1.0,3.0,3.0,5486.0,1.0
min,1.0,1.0,0.0,5.0,6.0,19.0,1.0,1.0,1.0,7000.0,1.0
25%,2.0,1.0,0.0,7.0,20.0,38.0,1.0,2.0,4.0,8500.0,1.0
50%,4.0,1.0,0.0,9.0,30.0,49.0,2.0,3.0,5.0,10000.0,2.0
75%,5.0,1.0,1.0,11.0,41.0,63.0,2.0,5.0,9.0,13125.0,3.0
max,6.0,1.0,3.0,81.0,303.0,403.0,4.0,11.0,16.0,71000.0,3.0


median price for [*status* == 1] is 10000

In [47]:
full_data.loc[full_data.status == 2].describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,2736.0,2736.0,2736.0,1347.0,890.0,2736.0,2736.0,2736.0,2736.0,2736.0,2736.0
mean,4.0,2.0,1.0,18.0,37.0,66.0,2.0,5.0,8.0,17809.0,3.0
std,2.0,0.0,1.0,72.0,34.0,252.0,1.0,3.0,4.0,6606.0,1.0
min,1.0,2.0,0.0,3.0,8.0,7.0,1.0,1.0,1.0,7000.0,1.0
25%,2.0,2.0,0.0,10.0,21.0,44.0,1.0,2.0,5.0,14000.0,3.0
50%,3.0,2.0,1.0,13.0,32.0,54.0,2.0,4.0,9.0,16500.0,3.0
75%,5.0,2.0,1.0,16.0,42.0,70.0,2.0,7.0,10.0,20300.0,3.0
max,6.0,2.0,4.0,1628.0,654.0,13000.0,7.0,21.0,25.0,76900.0,3.0


median price for [*status* == 2] is 16500

In [48]:
full_data.loc[full_data.status == 3].describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,1551.0,1551.0,1551.0,619.0,464.0,1551.0,1551.0,1551.0,1551.0,1551.0,1551.0
mean,3.0,3.0,1.0,18.0,37.0,68.0,2.0,5.0,9.0,22542.0,3.0
std,1.0,0.0,1.0,15.0,32.0,44.0,1.0,3.0,4.0,9719.0,0.0
min,1.0,3.0,0.0,3.0,10.0,15.0,1.0,1.0,1.0,7200.0,1.0
25%,2.0,3.0,0.0,12.0,20.0,45.0,1.0,3.0,5.0,16000.0,3.0
50%,3.0,3.0,1.0,15.0,34.0,60.0,2.0,4.0,9.0,20000.0,3.0
75%,5.0,3.0,1.0,20.0,42.0,78.0,2.0,7.0,10.0,25000.0,3.0
max,6.0,3.0,4.0,232.0,422.0,728.0,7.0,21.0,25.0,96100.0,3.0


median price for [*status* == 3] is 20000

Filling missed values *status* by median price

In [49]:
full_data.loc[full_data['status'].isnull() & (full_data['price'] <= 10000), 'status'] = 1
full_data.loc[full_data['status'].isnull() & (full_data['price'] <= 17000), 'status'] = 2
full_data.loc[full_data['status'].isnull(), 'status'] = 3

In [50]:
full_data.head()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
0,1,2.0,1.0,11.0,27.0,52,1,9,11,12500,3
1,1,1.0,2.0,6.0,42.0,58,3,5,5,8500,1
2,5,2.0,1.0,18.0,15.0,44,1,2,3,19300,2
3,5,2.0,1.0,NaN,NaN,48,1,8,10,15000,3
4,2,3.0,1.0,NaN,NaN,50,1,8,12,30800,3


#### Concat the data

data_march_1 is already preprocessed so just merge 

In [51]:
data_march_1.index = np.arange(len(full_data)+1, len(data_march_1) + len(full_data) + 1)
full_data = pd.concat([full_data, data_march_1])

In [52]:
full_data = full_data.drop_duplicates()

In [53]:
full_data.shape

(11065, 11)

In [54]:
full_data.describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,superficiality,price,age
count,11065.0,11065.0,11065.0,7015.0,6312.0,11065.0,11065.0,11065.0,6381.0,11065.0,11065.0
mean,3.0,2.0,0.0,16.0,38.0,66.0,2.0,5.0,8.0,19552.0,3.0
std,1.0,1.0,1.0,32.0,20.0,191.0,1.0,3.0,4.0,8516.0,1.0
min,1.0,1.0,0.0,3.0,6.0,7.0,1.0,0.0,1.0,1500.0,1.0
25%,2.0,2.0,0.0,14.0,26.0,45.0,1.0,2.0,5.0,14000.0,3.0
50%,3.0,2.0,0.0,15.0,39.0,58.0,2.0,4.0,9.0,17550.0,3.0
75%,5.0,3.0,1.0,16.0,42.0,72.0,2.0,7.0,10.0,23100.0,3.0
max,6.0,3.0,4.0,1628.0,654.0,15000.0,7.0,30.0,25.0,96100.0,3.0


In [55]:
full_data.isna().sum()

district             0
status               0
balcony              0
area_kitchen      4050
area_living       4753
area                 0
room                 0
floor                0
superficiality    4684
price                0
age                  0
dtype: int64

Column *superficiality* has more than 50 % missed values, so drop this column

In [56]:
full_data = full_data.drop(columns = ['superficiality'], axis = 1)

In [57]:
full_data.loc[full_data.area > 300]

,district,status,balcony,area_kitchen,area_living,area,room,floor,price,age
1549,5.0,2.0,2.0,15.0,55.0,1051.0,3,7,22700,3
2129,2.0,1.0,0.0,81.0,303.0,403.0,1,2,8000,2
2130,6.0,2.0,1.0,6.0,NaN,422.0,2,3,10000,2
2969,5.0,2.0,1.0,NaN,NaN,594.0,2,7,15000,1
3011,4.0,2.0,2.0,15.0,55.0,1051.0,3,7,22700,3
3341,5.0,2.0,0.0,NaN,NaN,700.0,2,7,27000,3
3562,5.0,2.0,0.0,7.0,294.0,364.0,1,9,11600,1
3906,5.0,2.0,1.0,75.0,20.0,379.0,1,3,25000,3
4610,2.0,3.0,0.0,NaN,NaN,376.0,1,4,19300,3
4677,2.0,2.0,1.0,7.0,NaN,13000.0,2,1,13000,2


In [58]:
full_data = full_data.loc[full_data.area < 200]

In [59]:
full_data = full_data.loc[full_data.price >= 7000]

In [60]:
full_data.describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,price,age
count,10979.0,10979.0,10979.0,6962.0,6263.0,10979.0,10979.0,10979.0,10979.0,10979.0
mean,3.0,2.0,0.0,16.0,37.0,62.0,2.0,5.0,19455.0,3.0
std,1.0,1.0,1.0,32.0,16.0,23.0,1.0,3.0,8263.0,1.0
min,1.0,1.0,0.0,3.0,6.0,7.0,1.0,0.0,7000.0,1.0
25%,2.0,2.0,0.0,14.0,26.0,45.0,1.0,2.0,14000.0,3.0
50%,3.0,2.0,0.0,15.0,39.0,57.0,2.0,4.0,17550.0,3.0
75%,5.0,3.0,1.0,16.0,42.0,72.0,2.0,7.0,23100.0,3.0
max,6.0,3.0,4.0,1628.0,444.0,180.0,7.0,30.0,90000.0,3.0


Filling missed data using same approach as with *status* column

In [61]:
full_data.loc[full_data.area <= 15, 'area_kitchen'] = full_data.loc[full_data.area <= 15, 'area_kitchen'].fillna(3)
full_data.loc[full_data.area <= 15, 'area_living'] = full_data.loc[full_data.area <= 15, 'area_living'].fillna(8)
full_data.loc[full_data.area <= 46, 'area_kitchen'] = full_data.loc[full_data.area <= 46, 'area_kitchen'].fillna(15)
full_data.loc[full_data.area <= 46, 'area_living'] = full_data.loc[full_data.area <= 46, 'area_living'].fillna(27)
full_data.loc[full_data.area <= 59, 'area_kitchen'] = full_data.loc[full_data.area <= 59, 'area_kitchen'].fillna(15)
full_data.loc[full_data.area <= 59, 'area_living'] = full_data.loc[full_data.area <= 59, 'area_living'].fillna(39)
full_data.loc[full_data.area > 59, 'area_kitchen'] = full_data.loc[full_data.area > 59, 'area_kitchen'].fillna(16)
full_data.loc[full_data.area > 59 , 'area_living'] = full_data.loc[full_data.area > 59, 'area_living'].fillna(42)

In [62]:
full_data.shape

(10979, 10)

In [63]:
full_data.describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,price,age
count,10979.0,10979.0,10979.0,10979.0,10979.0,10979.0,10979.0,10979.0,10979.0,10979.0
mean,3.0,2.0,0.0,16.0,37.0,62.0,2.0,5.0,19455.0,3.0
std,1.0,1.0,1.0,25.0,13.0,23.0,1.0,3.0,8263.0,1.0
min,1.0,1.0,0.0,3.0,6.0,7.0,1.0,0.0,7000.0,1.0
25%,2.0,2.0,0.0,15.0,27.0,45.0,1.0,2.0,14000.0,3.0
50%,3.0,2.0,0.0,15.0,39.0,57.0,2.0,4.0,17550.0,3.0
75%,5.0,3.0,1.0,16.0,42.0,72.0,2.0,7.0,23100.0,3.0
max,6.0,3.0,4.0,1628.0,444.0,180.0,7.0,30.0,90000.0,3.0


#### Work with outliers

In [64]:
full_data = full_data.loc[full_data.area_kitchen < 100]

In [65]:
full_data = full_data.loc[full_data.area_living <= 200]

In [67]:
full_data.describe().round()

,district,status,balcony,area_kitchen,area_living,area,room,floor,price,age
count,10960.0,10960.0,10960.0,10960.0,10960.0,10960.0,10960.0,10960.0,10960.0,10960.0
mean,3.0,2.0,0.0,15.0,37.0,62.0,2.0,5.0,19445.0,3.0
std,1.0,1.0,1.0,4.0,11.0,23.0,1.0,3.0,8259.0,1.0
min,1.0,1.0,0.0,3.0,6.0,7.0,1.0,0.0,7000.0,1.0
25%,2.0,2.0,0.0,15.0,27.0,45.0,1.0,2.0,14000.0,3.0
50%,3.0,2.0,0.0,15.0,39.0,57.0,2.0,4.0,17550.0,3.0
75%,5.0,3.0,1.0,16.0,42.0,72.0,2.0,7.0,23100.0,3.0
max,6.0,3.0,4.0,83.0,175.0,180.0,7.0,30.0,90000.0,3.0


#### Save ready dataset

In [70]:
# uncomment if need it 
# full_data.to_csv('after_preprocessing.csv', encoding='utf-8', index=False)